In [1]:
from mhcflurry import Class1AffinityPredictor
from json import loads, dumps
import pandas

Using TensorFlow backend.


In [2]:
alleles_of_interest = ['HLA-A*01:01', 'HLA-A*02:01', 'HLA-A*02:03', 'HLA-A*03:01', 'HLA-A*11:01', 'HLA-A*24:02', 'HLA-A*29:02', 'HLA-A*31:01', 'HLA-A*68:02', 'HLA-B*07:02', 'HLA-B*15:01', 'HLA-B*35:01', 'HLA-B*44:02', 'HLA-B*44:03', 'HLA-B*51:01', 'HLA-B*54:01', 'HLA-B*57:01']
model_of_interest = "KGX7QIJXWU"

In [3]:
best_trained_model = Class1AffinityPredictor.load()
just_trained_model = Class1AffinityPredictor.load('./'+model_of_interest)

In [4]:
df = just_trained_model.manifest_df.copy()
df["config"] = df.config_json.map(loads)
#df.config.iloc[0]

In [5]:
d = dict((v, i) for (i, v) in enumerate(sorted(set([
    dumps(v['hyperparameters']) for v in df.config
          ]))))

In [6]:
df["architecture_num"] = df.config.map(lambda v: d[dumps(v['hyperparameters'])])

In [7]:
df.architecture_num.value_counts()

0    17
Name: architecture_num, dtype: int64

In [12]:
def ppv(is_hit, predictions):
    df = pandas.DataFrame({"prediction": predictions, "is_hit": is_hit})
    return df.sort_values("prediction", ascending=True)[:int(is_hit.sum())].is_hit.mean()    

In [9]:
mass_spec_df = pandas.read_csv("abelin_peptides.mhcflurry.csv.bz2")
model_df = pandas.read_csv("KGX7QIJXWU-out.csv")

In [10]:
print(len(mass_spec_df))
print(len(model_df))

mass_spec_df = mass_spec_df[mass_spec_df.allele.isin(alleles_of_interest)]
model_df = model_df[model_df.allele.isin(alleles_of_interest)]
print(len(mass_spec_df))
print(len(model_df))

2566814
2566814
2065551
2065551


In [13]:
print(ppv(mass_spec_df.hit.values, mass_spec_df.mhcflurry.values))
print(ppv(model_df.hit.values, model_df.KGX7QIJXWUprediction))

0.6295046696983032
0.5343992958779522


In [38]:
model_groups = model_df.groupby('allele')
mass_spec_groups = mass_spec_df.groupby('allele')
mass_spec_ppv = mass_spec_groups.apply(lambda mini_df: (ppv(mini_df.hit.values, mini_df.mhcflurry.values), len(mini_df) ) )
model_ppv = model_groups.apply(lambda mini_df: ( ppv(mini_df.hit.values, mini_df.KGX7QIJXWUprediction), len(mini_df) ) )
print(model_ppv)
print(mass_spec_ppv)

allele
HLA-A*01:01     (0.8068635275339185, 126553)
HLA-A*02:01     (0.6554878048780488, 265024)
HLA-A*02:03     (0.6513911620294599, 185133)
HLA-A*03:01     (0.5925480769230769, 168064)
HLA-A*24:02     (0.7303167420814479, 223210)
HLA-A*29:02       (0.700228832951945, 88274)
HLA-A*31:01     (0.4926829268292683, 124230)
HLA-A*68:02    (0.48012078510317063, 200687)
HLA-B*35:01      (0.5825892857142857, 90496)
HLA-B*44:02     (0.6961789375582479, 108373)
HLA-B*44:03      (0.7393209200438116, 92213)
HLA-B*51:01     (0.6438645980253879, 143218)
HLA-B*54:01     (0.6254125412541254, 122412)
HLA-B*57:01     (0.6147151898734177, 127664)
dtype: object
allele
HLA-A*01:01    (0.8260175578611333, 126553)
HLA-A*02:01    (0.6745426829268293, 265024)
HLA-A*02:03    (0.7364975450081833, 185133)
HLA-A*03:01    (0.6009615384615384, 168064)
HLA-A*24:02    (0.7760180995475113, 223210)
HLA-A*29:02     (0.7139588100686499, 88274)
HLA-A*31:01    (0.4983739837398374, 124230)
HLA-A*68:02    (0.5067941620533467

In [30]:
import numpy as np
g = pandas.DataFrame(['A','B','A','C','D','D','E'])
gg = g.groupby(0)
print(gg)
histo = gg.apply(lambda x: x.count())
histo[1] = histo.astype(np.float)/len(g) 
print(histo)


   0         1
0             
A  2  0.285714
B  1  0.142857
C  1  0.142857
D  2  0.285714
E  1  0.142857


In [28]:
from json import loads
from mhcflurry.hyperparameters import HyperparameterDefaults
hyperparameters = open("hyperparameters.json").read()
hyperparameters = loads(hyperparameters)
combinations_dict = dict(layer_sizes=[ [16], [32], [64]  ],
                         dense_layer_l1_regularization=[ 0, .001, 0.01],
                         locally_connected_layers=[
                            [{'filters': 8, 'activation': 'tanh', 'kernel_size': 3}],
                            [{'filters': 8, 'activation': 'tanh', 'kernel_size': 3}, {'filters': 8, 'activation': 'tanh', 'kernel_size': 3}],
                            [],
                         ],
                         random_negative_constant=[0, 20])
hyperparameters_obj = HyperparameterDefaults(**hyperparameters[0])

print(hyperparameters_obj.models_grid(**combinations_dict))


{'n_models': 8, 'max_epochs': 500, 'patience': 10, 'early_stopping': True, 'validation_split': 0.2, 'random_negative_rate': 0.0, 'random_negative_constant': 25, 'use_embedding': False, 'kmer_size': 15, 'batch_normalization': False, 'locally_connected_layers': [{'filters': 8, 'activation': 'tanh', 'kernel_size': 3}, {'filters': 8, 'activation': 'tanh', 'kernel_size': 3}], 'activation': 'relu', 'output_activation': 'sigmoid', 'layer_sizes': [32], 'random_negative_affinity_min': 20000.0, 'random_negative_affinity_max': 50000.0, 'dense_layer_l1_regularization': 0.001, 'dropout_probability': 0.0}
[{'n_models': 8, 'max_epochs': 500, 'patience': 10, 'early_stopping': True, 'validation_split': 0.2, 'random_negative_rate': 0.0, 'random_negative_constant': 0, 'use_embedding': False, 'kmer_size': 15, 'batch_normalization': False, 'locally_connected_layers': [{'filters': 8, 'activation': 'tanh', 'kernel_size': 3}], 'activation': 'relu', 'output_activation': 'sigmoid', 'layer_sizes': [16], 'random_